In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup

In [2]:
# function for downloading content from the web
def content_downloader(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, "lxml") 
    return str(soup.find("div", {"class": "postArticle-content js-postField js-notesSource js-trackedPost"}))

In [3]:
# cleaning the content and return sentences it's not compelete and need to be modified
def cleaning_text(html_part):
    text = re.sub('<[A-Za-z\/][^>]*>', ' ', str(html_part))
    text = re.split(r'\s{2,}', text)[1:-1]
    return text

In [9]:
# preparing a text for execute caculations on it
def prepare_text_from_html(url):
    html_content = content_downloader(url)
    text = cleaning_text(html_content)
    return text

In [5]:
# function for loading diffrenet module
def loading_module(module_url):
    # Import the Universal Sentence Encoder's TF Hub module
    embed_object = hub.Module(module_url)
    return embed_object

In [6]:
# function for runinng embedding module on text
def run_embedding(embed_object, text):
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed_object(text))

    return message_embeddings

In [7]:
# function for calculating similarity between question and text
def calculating_text_similarity(question, text, module_url):
    question_tensor = tf.Variable(tf.convert_to_tensor(run_embedding(loading_module(module_url), question)))
    text_tensor = tf.Variable(tf.convert_to_tensor(run_embedding(loading_module(module_url), text)))
    multiply_tensor = tf.matmul(question_tensor, text_tensor, transpose_b = True)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        answer = sess.run(multiply_tensor)
    return answer

In [8]:
# function for find sentence in text that answer question that has been asked
def find_similar_sentence(similarity_tensor, question, text):
    text = np.array(text)
    question_and_text = np.hstack([question, text]).reshape(-1, 1)
    print(question_and_text)
    return text[np.where(similarity_tensor > 0.4)[0]]

In [ ]:
text = prepare_text_from_html('https://towardsdatascience.com/5-resources-to-inspire-your-next-data-science-project-ea6afbe20319')

In [ ]:
print(run_embedding(loading_module('https://tfhub.dev/google/universal-sentence-encoder/2'), [text[0]]))

In [ ]:
# def calculate_different_model_accuracy(question, text, module_url1, module_url2):